In [2]:
import malariagen_data


In [3]:
ag3 = malariagen_data.Ag3(pre=True)

In [16]:
ag3.biallelic_snps_to_plink(results_dir='/Users/dennistpw/Projects/malariagen-data-python/',
                            region='2L:100000-2000000',
                            n_snps=2000,
                            sample_sets='AG1000G-AO',
                            )

Subsetting to segregating sites: ⠏ (0:00:01.62) count alleles
Subsetting to segregating sites: ⠴ (0:00:02.14) ascertain segregating  sites
Subsetting to segregating sites: ⠦ (0:00:02.22) ascertained 1,089,159 sites
thin sites                                      
thinned to 2,002 sites
Set up dataset
[########################################] | 100% Completed | 103.94 ms
Ensure genotypes vary                         
final no. variants 138
Load final data
write plink files to /Users/dennistpw/Projects/malariagen-data-python//2L:100000-2000000.2000.0.0.0


'/Users/dennistpw/Projects/malariagen-data-python//2L:100000-2000000.2000.0.0.0'

In [5]:
from collections import Counter
from typing import Optional, Tuple

import allel  # type: ignore
import numpy as np
import os
#import bed_reader
from dask.diagnostics import ProgressBar
# import bokeh.plotting


In [6]:
import allel
region='2L:100000-2000000'
n_snps=2000
sample_sets='AG1000G-AO'
ds_snps = ag3.snp_calls(
            sample_sets='AG1000G-AO',
            region=region)

In [ ]:
ds_snps['variant_allele'].compute()

<xarray.DataArray 'variant_allele' (variants: 1677564, alleles: 4)>
array([[b'C', b'A', b'T', b'G'],
       [b'G', b'A', b'C', b'T'],
       [b'C', b'A', b'T', b'G'],
       ...,
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G'],
       [b'A', b'C', b'T', b'G']], dtype='|S1')
Coordinates:
    variant_position  (variants) int32 100000 100001 100002 ... 1999999 2000000
    variant_contig    (variants) uint8 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
Dimensions without coordinates: variants, alleles

In [7]:
gt = ds_snps["call_genotype"].data.compute()
print("count alleles")
with ProgressBar():
    ac = allel.GenotypeArray(gt).count_alleles(max_allele=3)
    print("ascertain segregating  sites")
    n_chroms = ds_snps.dims["samples"] * 2
    an_called = ac.sum(axis=1)
    an_missing = n_chroms - an_called
    min_ref_ac = 0
    max_ref_ac = n_chroms - 0
    # here we choose segregating sites
    loc_sites = (
        (ac[:, 0] >= min_ref_ac)
        & (ac[:, 0] <= max_ref_ac)
        & (an_missing <= 0)
    )
print(f"ascertained {np.count_nonzero(loc_sites):,} sites")


count alleles
ascertain segregating  sites
ascertained 1,089,159 sites


In [8]:
#set up dataset with required vars for plink conversion
print("Set up dataset")
ds_snps_asc = (
    ds_snps[["variant_contig", "variant_position", "variant_allele", "sample_id", "call_genotype"]]
    .isel(alleles=slice(0, 2))
    .sel(variants=loc_sites)
    )

#compute gt ref counts
gt_asc = ds_snps_asc["call_genotype"].data.compute()
gn_ref = allel.GenotypeDaskArray(gt_asc).to_n_ref(fill=-127) #what does the fill do
with ProgressBar():
    gn_ref = gn_ref.compute()

print("Ensure genotypes vary")
loc_var = np.any(gn_ref != gn_ref[:, 0, np.newaxis], axis=1)
print(f"final no. variants {np.count_nonzero(loc_var)}")


print("Load final data")
with ProgressBar():
    ds_snps_final = (
        ds_snps_asc[["variant_contig", "variant_position", "variant_allele", "sample_id"]]
        .isel(variants=loc_var)
    )

Set up dataset
[########################################] | 100% Completed | 698.99 ms
Ensure genotypes vary
final no. variants 67780
Load final data


In [47]:
def apply_allele_mapping(x, mapping, max_allele):
    """Transform an array x, where the columns correspond to alleles,
    according to an allele mapping.

    Values from columns in x will be move to different columns in the
    output according to the allele index mapping given.

    """
    n_sites = x.shape[0]
    n_alleles = x.shape[1]
    assert mapping.shape[0] == n_sites
    assert mapping.shape[1] == n_alleles

    # Create output array.
    out = np.empty(shape=(n_sites, max_allele + 1), dtype=x.dtype)

    # Iterate over sites.
    for i in range(n_sites):
        # Iterate over alleles.
        for allele_index in range(n_alleles):
            # Find the new index for this allele.
            new_allele_index = mapping[i, allele_index]

            # Copy data to the new allele index.
            if new_allele_index >= 0 and new_allele_index <= max_allele:
                out[i, new_allele_index] = x[i, allele_index]

    return out

def trim_alleles(ac):
    """Remap allele indices to trim out unobserved alleles.

    The idea here is that our SNP data includes alleles which
    may not ever be observed, or may not be observed within a
    particular subset of the data. For convenience, it is useful
    to retain only alleles which are observed.

    Here we use a given set of allele counts (ac) to identify
    observed alleles and derive a mapping from old allele
    indices to a new set of allele indices.

    For example, if alleles 0 and 2 are observed at a given
    site in ac, then 0 will be mapped to 0 and 2 will be mapped
    to 1.

    Similarly, if alleles 1 and 3 are observed at a given site
    in ac, then 1 will be mapped to 0 and 3 will be mapped to 1.

    """

    n_variants = ac.shape[0]
    n_alleles = ac.shape[1]

    # Create the output array - this is an allele mapping array,
    # that specifies how to recode allele indices.
    mapping = np.empty((n_variants, n_alleles), dtype=np.int32)

    # The value of -1 indicates that there is no mapping for
    # a given allele. We fill with -1 so we can then set
    # values where we do observe alleles and can define a
    # mapping.
    mapping[:] = -1

    # Iterate over variants.
    for i in range(n_variants):
        # This will be the new index that we are mapping this allele to, if the
        # allele is observed.
        new_allele_index = 0

        # Iterate over columns (alleles) in the input array.
        for allele_index in range(n_alleles):
            # Access the count for the jth allele.
            c = ac[i, allele_index]
            if c > 0:
                # We have found a non-zero allele count, remap the allele.
                mapping[i, allele_index] = new_allele_index

                # Increment to be ready for the next allele.
                new_allele_index += 1

    return mapping

In [48]:
region='2L:1001000-2009000',


In [49]:
ds = ag3.snp_calls(
    region=region,
    sample_sets=sample_sets
)

In [51]:
# Perform an allele count.
ac = ag3.snp_allele_counts(
    region=region,
    sample_sets=sample_sets
)

# Locate biallelic SNPs.
loc_bi = allel.AlleleCountsArray(ac).is_biallelic()

# Remap alleles to squeeze out unobserved alleles.
ac_bi = ac[loc_bi]
allele_mapping = trim_alleles(ac_bi)

Compute SNP allele counts:   0%|          | 0/20 [00:00<?, ?it/s]

In [52]:
ds_bi = ds.isel(variants=loc_bi)


In [53]:
from typing import Any, Dict, List, Optional, Tuple, Union


In [54]:
# Start building a new dataset.
coords: Dict[str, Any] = dict()
data_vars: Dict[str, Any] = dict()

In [56]:
# Store sample IDs.
coords["sample_id"] = ("samples",), ds_bi["sample_id"].data

# Store contig.
coords["variant_contig"] = ("variants",), ds_bi["variant_contig"].data

# Store position.
coords["variant_position"] = ("variants",), ds_bi["variant_position"].data

In [1]:
import dask.array as da
variant_allele = ds_bi["variant_allele"].data
variant_allele = variant_allele.rechunk((variant_allele.chunks[0], -1))
variant_allele_out = da.map_blocks(
    lambda block: apply_allele_mapping(block, allele_mapping, max_allele=1),
    variant_allele,
    dtype=variant_allele.dtype,
    chunks=(variant_allele.chunks[0], [2]),
)
data_vars["variant_allele"] = ("variants", "alleles"), variant_allele_out

# Store allele counts, transformed, so we don't have to recompute.
#ac_out = apply_allele_mapping(ac_bi, allele_mapping, max_allele=1)
#data_vars["variant_allele_count"] = ("variants", "alleles"), ac_out

NameError: name 'ds_bi' is not defined

In [64]:
variant_allele = ds_bi["variant_allele"].data
variant_allele = variant_allele.rechunk((variant_allele.chunks[0], -1))

In [67]:
variant_allele.compute()

array([[b'T', b'A', b'C', b'G'],
       [b'G', b'A', b'C', b'T'],
       [b'T', b'A', b'C', b'G'],
       ...,
       [b'G', b'A', b'C', b'T'],
       [b'C', b'A', b'T', b'G'],
       [b'A', b'C', b'T', b'G']], dtype='|S1')

In [32]:
variant_allele = variant_allele.rechunk((variant_allele.chunks[0], -1))


array([[b'T', b'A', b'C', b'G'],
       [b'G', b'A', b'C', b'T'],
       [b'T', b'A', b'C', b'G'],
       ...,
       [b'G', b'A', b'C', b'T'],
       [b'C', b'A', b'T', b'G'],
       [b'A', b'C', b'T', b'G']], dtype='|S1')

In [ ]:
   ds = self.snp_calls(
            region=region,
            sample_sets=sample_sets,
            sample_query=sample_query,
            sample_indices=sample_indices,
            site_mask=site_mask,
            site_class=site_class,
            cohort_size=cohort_size,
            min_cohort_size=min_cohort_size,
            max_cohort_size=max_cohort_size,
            random_seed=random_seed,
            inline_array=inline_array,
            chunks=chunks,
        )

        with self._spinner("Prepare biallelic SNP calls"):
            # Subset to biallelic sites.
            ds_bi = ds.isel(variants=loc_bi)

            # Start building a new dataset.
            coords: Dict[str, Any] = dict()
            data_vars: Dict[str, Any] = dict()

            # Store sample IDs.
            coords["sample_id"] = ("samples",), ds_bi["sample_id"].data

            # Store contig.
            coords["variant_contig"] = ("variants",), ds_bi["variant_contig"].data

            # Store position.
            coords["variant_position"] = ("variants",), ds_bi["variant_position"].data

            # Store alleles, transformed.
            variant_allele = ds_bi["variant_allele"].data
            variant_allele = variant_allele.rechunk((variant_allele.chunks[0], -1))
            variant_allele_out = da.map_blocks(
                lambda block: apply_allele_mapping(block, allele_mapping, max_allele=1),
                variant_allele,
                dtype=variant_allele.dtype,
                chunks=(variant_allele.chunks[0], [2]),
            )
            data_vars["variant_allele"] = ("variants", "alleles"), variant_allele_out

            # Store allele counts, transformed, so we don't have to recompute.
            ac_out = apply_allele_mapping(ac_bi, allele_mapping, max_allele=1)
            data_vars["variant_allele_count"] = ("variants", "alleles"), ac_out

            # Store genotype calls, transformed.
            gt = ds_bi["call_genotype"].data
            gt_out = allel.GenotypeDaskArray(gt).map_alleles(allele_mapping)
            data_vars["call_genotype"] = (
                (
                    "variants",
                    "samples",
                    "ploidy",
                ),
                gt_out.values,
            )